In [1]:
import types
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
# PyTorch
import torch
import torchvision
import torchmetrics

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import losses
import utils

In [3]:
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
n = 1000
tune = False
random_state = 1001
augmented_train_dataset, train_dataset, val_or_test_dataset = utils.get_cifar10_datasets(dataset_directory, n, tune, random_state)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
batch_size = 128
num_workers = 0
augmented_train_loader = torch.utils.data.DataLoader(augmented_train_dataset, batch_size=min(batch_size, len(augmented_train_dataset)), shuffle=True, num_workers=num_workers, drop_last=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=min(batch_size, len(train_dataset)), num_workers=num_workers)
val_or_test_loader = torch.utils.data.DataLoader(val_or_test_dataset, batch_size=batch_size, num_workers=num_workers)

In [5]:
num_classes = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = torchvision.models.resnet50()
model.fc = torch.nn.Linear(in_features=2048, out_features=num_classes, bias=True)
model.to(device)

bb_loc = torch.load('/cluster/tufts/hugheslab/eharve06/resnet50_torchvision/resnet50_torchvision_mean.pt', map_location=torch.device('cpu')).to(device)
criterion = losses.ERMLoss()

cuda:0


In [6]:
checkpoint = torch.load('/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_VI/l2-sp_kappa=23528.522_lr_0=0.01_n=1000_random_state=1001.pt', map_location=torch.device('cpu'))

checkpoint = {key.replace('.layer', ''): value for key, value in checkpoint.items()}

sigma_param = checkpoint['sigma_param']

keys_to_remove = [key for key in checkpoint if key.endswith('sigma_param')]
for key in keys_to_remove:
    checkpoint.pop(key, None)
    
for key in checkpoint:
    if checkpoint[key].requires_grad:
        checkpoint[key] += torch.nn.functional.softplus(sigma_param) * torch.randn_like(checkpoint[key])

model.load_state_dict(checkpoint)
model.to(device)

val_or_test_metrics = utils.evaluate(model, criterion, val_or_test_loader, num_classes=num_classes)
print(val_or_test_metrics['acc'])
print(val_or_test_metrics['nll'])

0.8927000761032104
0.8442955075263979


In [7]:
utils.bn_update(model, train_loader)
val_or_test_metrics = utils.evaluate(model, criterion, val_or_test_loader, num_classes=num_classes)
print(val_or_test_metrics['acc'])
print(val_or_test_metrics['nll'])

0.8985000252723694
0.8001144462585451
